In [7]:
import pandas as pd

In [8]:
def merge_dataframes(user_health_df, supplement_usage_df, experiments_df, user_profiles_df):
    return supplement_usage_df.merge(experiments_df, on='experiment_id')\
        .merge(user_health_df, how='outer', on=['user_id', 'date'])\
        .merge(user_profiles_df, how='left', on='user_id')

In [9]:
def clean_dataframe(merged_df):
    merged_df['date'] = pd.to_datetime(merged_df['date'])

    bins = [0, 17, 25, 35, 45, 55, 65, float('inf')]
    labels = ['Under 18', '18-25', '26-35', '36-45', '46-55', '56-65', 'Over 65']
    merged_df['user_age_group'] = pd.cut(merged_df['age'], bins=bins, labels=labels)\
        .cat.add_categories('Unknown').fillna('Unknown')

    merged_df['supplement_name'] = merged_df['supplement_name'].fillna('No intake')

    merged_df['dosage_grams'] = merged_df['dosage'] / 1000

    merged_df['sleep_hours'] = merged_df['sleep_hours'].str.lower().str.replace('h', '').astype(float)

    merged_df['is_placebo'] = merged_df['is_placebo'].astype(bool)
    
    merged_df.rename(columns={'name': 'experiment_name'}, inplace=True)
    merged_df.drop(columns=['dosage', 'description', 'dosage_unit', 'experiment_id', 'age'], inplace=True)
    
    return merged_df

In [10]:
def merge_and_clean_all_data(user_health_file_path, supplement_usage_file_path, experiments_file_path, user_profiles_file_path):
    user_health_df = pd.read_csv(user_health_file_path)
    supplement_usage_df = pd.read_csv(supplement_usage_file_path)
    experiments_df = pd.read_csv(experiments_file_path)
    user_profiles_df = pd.read_csv(user_profiles_file_path)

    merged_df = merge_dataframes(user_health_df, supplement_usage_df, experiments_df, user_profiles_df)
    clean_df = clean_dataframe(merged_df)
    
    return clean_df

In [11]:
df = merge_and_clean_all_data(
    'data/user_health_data.csv',
    'data/supplement_usage.csv',
    'data/experiments.csv',
    'data/user_profiles.csv'
)

df.head(10)

,user_id,date,supplement_name,is_placebo,experiment_name,average_heart_rate,average_glucose,sleep_hours,activity_level,email,user_age_group,dosage_grams
0,005dd64e-3863-49f3-93b0-1aea9a84d526,2018-01-31,Placebo,False,Memory,84.172177,96.814856,11.4,2,user_65@myemail.com,26-35,0.170989
1,005dd64e-3863-49f3-93b0-1aea9a84d526,2018-02-28,Magnesium,False,Sleep Quality,85.200747,130.836935,5.8,2,user_65@myemail.com,26-35,0.494938
2,005dd64e-3863-49f3-93b0-1aea9a84d526,2018-03-31,Placebo,False,Sleep Quality,76.695318,109.782341,7.0,1,user_65@myemail.com,26-35,0.184596
3,005dd64e-3863-49f3-93b0-1aea9a84d526,2018-04-30,Omega-3,True,Endurance,65.772482,100.765545,7.5,1,user_65@myemail.com,26-35,0.313256
4,016e4095-8b5d-43d9-83fd-e28b38be2f7d,2018-01-31,No intake,True,NaN,86.762141,103.689177,8.2,2,contact_390@email.com,36-45,NaN
5,016e4095-8b5d-43d9-83fd-e28b38be2f7d,2018-02-28,Magnesium,False,Recovery,65.051339,113.214798,4.5,2,contact_390@email.com,36-45,0.226029
6,016e4095-8b5d-43d9-83fd-e28b38be2f7d,2018-02-28,Placebo,False,Endurance,65.051339,113.214798,4.5,2,contact_390@email.com,36-45,0.481695
7,016e4095-8b5d-43d9-83fd-e28b38be2f7d,2018-03-31,Magnesium,False,Strength,97.831060,98.343757,7.0,4,contact_390@email.com,36-45,0.242708
8,016e4095-8b5d-43d9-83fd-e28b38be2f7d,2018-04-30,Zinc,False,Strength,65.302963,83.886078,11.4,3,contact_390@email.com,36-45,0.360710
9,021fec0e-28e1-4f21-9fa3-c714703f0d7b,2018-01-31,No intake,True,NaN,65.003556,108.979109,12.0,4,user_481@email.com,Over 65,NaN
